NLP 처리과정 
토큰화 -> 정체,추출 -> 인코딩

# 1.표준 토큰화
자연어 처리에 사용되는 대표적인 파이썬 패키지는 NLTK

1.1 표준 토큰화
treebank 사용 


In [ ]:
!pip install nltk

In [ ]:
from nltk.tokenize import TreebankWordTokenizer
tokenizer  = TreebankWordTokenizer()
text = "Model-based RL don't need a value function for the policy"
print(tokenizer.tokenize(text))

['Model-based', 'RL', 'do', "n't", 'need', 'a', 'value', 'function', 'for', 'the', 'policy']


1.2 토큰화 라이브러리
여러종류의 tokenizer 가 있음 word_tokenizer

In [ ]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [ ]:
from nltk.tokenize import word_tokenize
print(word_tokenize(text))

['Model-based', 'RL', 'do', "n't", 'need', 'a', 'value', 'function', 'for', 'the', 'policy']


# 2. 어간 추출 및 표제어 추출
단어의 형태소 level에서 분석을 하게 되면 다른 품사 또는 다른 시제의 단어라고 해도 같은 형태로 토큰화 가능

-->둘의 차이 : 품사의 태깅 유무

-->표제어 : v, n 태깅 가능

-->어간 : 태깅 불가능

2.1 어간 추출Stemmer vs 표제어 추출lemmazation 

대표적 어간 추출 기법은 porter 추출 패키지

In [ ]:
#어간 추출
from nltk.stem import PorterStemmer, LancasterStemmer
stem1 = PorterStemmer()
stem2 = LancasterStemmer()
words = ["eat", "ate", "eaten", "eating"]
print("Poter Stemmer :", [stem1.stem(w) for w in words])
print("Lancaster Stemmer :", [stem2.stem(w) for w in words]) #왜 ate를 at으로..?

Poter Stemmer : ['eat', 'ate', 'eaten', 'eat']
Lancaster Stemmer : ['eat', 'at', 'eat', 'eat']


In [ ]:
#표제어 추출
from nltk import WordNetLemmatizer
nltk.download('wordnet')
lemm = WordNetLemmatizer()
words = ["eat", "ate", "eaten", "eating"]
print('WordNet Lemmetizer  :', [lemm.lemmatize(w, pos='v') for w in words]) #pos로 단어의 품사 지정해줌
#모든 시제를 동사원형으로 바꿔줌 품사태깅이 가능하다면 표제어 추출이 좋음

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
WordNet Lemmetizer  : ['eat', 'eat', 'eat', 'eat']


# 3. 불용어 제거
3.1불용어 예시

영어의 불용어 예시 

stopword단어 데이터를 받기 위한 사전작업이 필요함

In [ ]:
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
print(stopwords.words('english')[:5])

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
['i', 'me', 'my', 'myself', 'we']


In [ ]:
import nltk
nltk.download('punkt')
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

input_sentence = "we should all study hard for the exam."
stop_words = set(stopwords.words('english'))

#토큰화 후 불용어제거 for문

word_tokens = word_tokenize(input_sentence)
result = []
for w in word_tokens:
  if w not in stop_words:
    result.append(w)

print(word_tokens)
print(result)


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
['we', 'should', 'all', 'study', 'hard', 'for', 'the', 'exam', '.']
['study', 'hard', 'exam', '.']


# 4. 정수 인코딩 및 sorting
4.1 Enumerate 사용 

In [17]:
mylist = ['Enflish', 'math','Science' ]
for n, name in enumerate(mylist):
  print('Course : {}, Number : {}'.format(name,n))

  

Course : Enflish, Number : 0
Course : math, Number : 1
Course : Science, Number : 2


4.2 정수 인코딩 및 High-frequency Sorting

In [20]:
vocab = {'apple':2,'July':6,'piano':4,'cup':8,'orange':1} #BoW
vocab_sort = sorted(vocab.items(), key= lambda x:x[1], reverse = True) #슛자 기준 역순 정렬
print(vocab_sort)

# 많이 사용된 벡터에 1을 부여하고 싶음 -> word[0] = cup, index enumerate하면서 1씩 증가
word2inx = {word[0] : index +1 for index, word in enumerate(vocab_sort)}
print(word2inx)

[('cup', 8), ('July', 6), ('piano', 4), ('apple', 2), ('orange', 1)]
{'cup': 1, 'July': 2, 'piano': 3, 'apple': 4, 'orange': 5}


Bow로 만들어진 토큰화의 결과(Bow)를 가장 높은 빈도수부터 재정렬하고 이를 통해 정수 인코딩 진행

In [22]:
from nltk.tokenize import TreebankWordTokenizer
tokenizer = TreebankWordTokenizer()
text = "Model-based RL don't need a value function for the policy."\
       "but some of Model-based RL algorithms do have a value function."
token_text = tokenizer.tokenize(text)
word2inx = {}
Bow = []
for word in token_text:
  if word not in word2inx.keys():
    word2inx[word] = len(word2inx) #비어있는 상태의 딕셔너리 길이 즉 0부터 시작 ex> Model = 0 
    Bow.insert(len(word2inx)-1,1) #(0,1)에 insert
  else:
    inx= word2inx.get(word) #get함수의 의미 : 기존에 있는 word를 가져오라는 의미
    Bow[inx] +=1 #가져온 word값에 +1

print(word2inx) # inx
print(Bow) #빈도수를 고려해서 인코딩


{'Model-based': 0, 'RL': 1, 'do': 2, "n't": 3, 'need': 4, 'a': 5, 'value': 6, 'function': 7, 'for': 8, 'the': 9, 'policy.but': 10, 'some': 11, 'of': 12, 'algorithms': 13, 'have': 14, '.': 15}
[2, 2, 2, 1, 1, 2, 2, 2, 1, 1, 1, 1, 1, 1, 1, 1]


문장으로부터  토큰화를 통해 토큰 리스트를 만들고, 이를 이용해서 BoW를 생성하는 전체 알고리즘 word2inx = {}를 만들고, 리스트에 없는 단어의 경우 새로 리스트와 BoW에 단어를 추가하고 리스트에 있는 단어는 inx +=1

# 5. 유사도 분석

5.1코사인 유사도

In [28]:
import numpy as np
# 두 벡터 사이의 각도의 코사인값이 코사인 유사도 cf>cos0=1, cos90 = 0, cos180 = -1
# 내적 = a,b의 norm값 x cos0      *이때 내적은 각 성분끼리의 곱의 합산
def cos_sim(A,B):
  return np.dot(A,B) / (np.linalg.norm(A)*np.linalg.norm(B))

a = [1,0,0,1]
b = [0,1,1,0]
c = [1,1,1,1]
print(cos_sim(a,b), cos_sim(b,c), cos_sim(c,a))

0.0 0.7071067811865475 0.7071067811865475


5.2레반슈타인 거리

In [34]:
#어떻게하면 최소한의 수정단위를 거쳐서 처음단어에서 나중단어로 갈것인지 보여주는 거리
#추가, 삭제, 수정중 거리가 가장 짧은것을 return

def leven(text1, text2):
  len1 = len(text1) +1
  len2 = len(text2) +1

  sim_array = np.zeros((len1, len2)) #matrix테이블 만들기
  sim_array[:,0] = np.linspace(0, len1-1, len1) #모든행 첫열에 0~len1-1길이만큼 len1만큼 등분해서 숫자를 채워달라
  sim_array[0,:] = np.linspace(0, len2-1, len2) #0번째 행 모든 열에 마찬가지로 linspace만큼 채워달라
  for i in range(1, len1):
    for j in range(1, len2):
      add_char = sim_array[i-1,j] +1 # 추가는 위(row)의 값에 +1
      sub_char = sim_array[i,j-1] +1 # 삭제는 옆(column의 값에 +1 
      if text1[i-1] == text2[j-1]:
        mod_char = sim_array[i-1, j-1] #수정은 대각선에 있는것이 같으면 그대로, 다르면 +1 대각선 위치(i-1, j-1)
      else:
        mod_char = sim_array[i-1, j-1] + 1
      sim_array[i,j] = min([add_char, sub_char, mod_char]) # 추가, 삭제, 수정중 가장 짧은것을 return
  return sim_array[-1,-1]  # (-1,-1)의 위치 = array의 오른쪽 끝

print(leven('데이터마이닝', '데이타마닝'))

#3:09

2.0
